In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from dash import Dash, html, dcc, Input, Output, State
import os

# Load the dataset
real_estate_data = pd.read_csv("Real_Estate.csv")  # Replace with your actual file path

# Selecting features and target variable
features = ['Distance to the nearest MRT station', 'Number of convenience stores', 'Latitude', 'Longitude']
target = 'House price of unit area'

X = real_estate_data[features]
y = real_estate_data[target]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check if the model has been saved, otherwise train and save it
if not os.path.exists('linear_regression_model.pkl'):
    # Model initialization
    model = LinearRegression()
    
    # Training the model
    model.fit(X_train, y_train)
    
    # Saving the trained model
    joblib.dump(model, 'linear_regression_model.pkl')
else:
    # Load the saved model
    model = joblib.load('linear_regression_model.pkl')

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.Div([
        html.H1("Real Estate Price Prediction", style={'text-align': 'center'}),
        
        html.Div([
            dcc.Input(id='distance_to_mrt', type='number', placeholder='Distance to MRT Station (meters)',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='num_convenience_stores', type='number', placeholder='Number of Convenience Stores',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='latitude', type='number', placeholder='Latitude',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='longitude', type='number', placeholder='Longitude',
                      style={'margin': '10px', 'padding': '10px'}),
            html.Button('Predict Price', id='predict_button', n_clicks=0,
                        style={'margin': '10px', 'padding': '10px', 'background-color': '#007BFF', 'color': 'white'}),
        ], style={'text-align': 'center'}),
        
        html.Div(id='prediction_output', style={'text-align': 'center', 'font-size': '20px', 'margin-top': '20px'})
    ], style={'width': '50%', 'margin': '0 auto', 'border': '2px solid #007BFF', 'padding': '20px', 'border-radius': '10px'})
])



In [2]:
# Define callback to update output
@app.callback(
    Output('prediction_output', 'children'),
    [Input('predict_button', 'n_clicks')],
    [State('distance_to_mrt', 'value'), 
     State('num_convenience_stores', 'value'),
     State('latitude', 'value'),
     State('longitude', 'value')]
)
def update_output(n_clicks, distance_to_mrt, num_convenience_stores, latitude, longitude):
    if n_clicks > 0:
        if all(v is not None for v in [distance_to_mrt, num_convenience_stores, latitude, longitude]):
            try:
                # Prepare the feature vector
                features = pd.DataFrame([[distance_to_mrt, num_convenience_stores, latitude, longitude]], 
                                        columns=['Distance to the nearest MRT station', 'Number of convenience stores', 'Latitude', 'Longitude'])
                
                # Predict using the loaded model
                prediction = model.predict(features)[0]
                return f'Predicted House Price of Unit Area: {prediction:.2f}'
            except Exception as e:
                return f"Error in prediction: {str(e)}"
        else:
            return 'Please enter all values to get a prediction'
    return ''

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
